# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

cat: publications.tsv: No such file or directory


## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [4]:
publications = pd.read_csv("markdown_generator/publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2014-07-22,SPEED: the segmented pupil experiment for exop...,Ground-based and Airborne Telescopes V,NaN,"Patrice, Martinez & Olivier, Preis & Carole, G...",1,https://arxiv.org/pdf/1407.6826,NaN
1,2015-03-01,The SPEED Project: SPEEDing up Research and De...,The Messenger,An overview is presented of the Seg-\nmented P...,"Martinez, P. et al, The SPEED Project",2,https://escholarship.org/content/qt39h072gr/qt...,NaN
2,2015-01-01,Self-Coherent Camera as a focal plane phasing ...,Adaptive Optics for Extremely Large Telescopes...,In order to achieve the high performance requi...,"Janin-Potiron, P., Martinez, P., Baudoz, P., &...",3,https://escholarship.org/content/qt5vv5p369/qt...,NaN
3,2015-01-01,System analysis of the Segmented Pupil Experim...,Adaptive Optics for Extremely Large Telescopes...,SPEED is a new experiment in progress at the L...,"Preis, O.; Martinez, P.; Gouvret, C.; Dejonghe...",4,https://escholarship.org/content/qt39h072gr/qt...,NaN
4,2016-05-23,The self-coherent camera as a focal plane fine...,Astronomy & Astrophysics,Context. Direct imaging of Earth-like exoplane...,"P. Janin-Potiron, P. Martinez, P. Baudoz and M...",5,https://www.aanda.org/articles/aa/pdf/2016/08/...,NaN
5,2016-07-27,The Segmented Pupil Experiment for Exoplanet D...,Ground-based and Airborne Telescopes VI,The SPEED project - the Segmented Pupil Experi...,"P. Martinez, M. Beaulieu, P. Janin-Potiron, O....",6,,NaN
6,2016-01-01,The self-coherent camera as a focal plane phas...,EAS Publications Series,NaN,NaN,7,,NaN
7,2016-07-27,The self-coherent camera-phasing sensor: from ...,Ground-based and Airborne Telescopes VI,Extremely Large Telescopes (ELTs) are the next...,"P. Janin-Potiron, P. Martinez, P. Baudoz, M. C...",8,,NaN
8,2016-09-01,Laser-guide-stars used for cophasing broad cap...,Astronomy & Astrophysics,Context. Segmented primary mirrors are indispe...,"P. Martinez and P. Janin-Potiron\nA&A, 593 (20...",9,https://www.aanda.org/articles/aa/pdf/2016/09/...,NaN
9,2017-07-01,Fine cophasing of segmented aperture telescope...,Astronomy & Astrophysics,Context. Segmented aperture telescopes require...,"P. Janin-Potiron, M. N’Diaye, P. Martinez, A. ...",10,https://www.aanda.org/articles/aa/pdf/2017/07/...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [ ]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

TypeError: 'float' object is not iterable

These files are in the publications directory, one directory below where we're working from.

In [15]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md  2015-10-01-paper-title-number-3.md
2010-10-01-paper-title-number-2.md  2024-02-17-paper-title-number-4.md


In [16]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'http://academicpages.github.io/files/slides1.pdf'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download slides here](http://academicpages.github.io/files/slides1.pdf)

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).